In [47]:
# Load packages.
import os
import sys
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np
import json

import tqdm
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs


import wandb

os.environ['CURL_CA_BUNDLE'] = ''



In [48]:
key = json.load(open('D:\similarity-engine\wandb_config.json'))
wandb.login(key=key['key'])

wandb: Currently logged in as: simpleparadox. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\rsaha/.netrc
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.htm

True

In [49]:
# # Load data
# data = pd.read_parquet("../data/Processed_records.parquet")
# data = data.dropna()
# data.head()


In [50]:
# 
df = pd.read_csv("df_training_full.csv")

In [51]:
# Divide the dataset into training and test sets.
train_set = df.sample(frac=0.9, random_state=42)  # Fixing the seed to 42 to reproducibility.
test_set = df.drop(train_set.index)

In [52]:
# # Create a dataframe from simpletransformers. To fine-tune a language model, each sample should be a row in a text file.
# # Store the 'metadata_en_processed' column in a text file.
# with open('../data/simpletransformer_lm_train.txt', 'w') as f:
#     for item in train_set['metadata_en_processed']:
#         f.write("%s\n" % item)

# # Store the test set in a text file.
# with open('../data/simpletransformer_lm_test.txt', 'w') as f:
#     for item in test_set['metadata_en_processed']:
#         f.write("%s\n" % item)



## Simpletransformers code for sanity check.

In [54]:
model_args = LanguageModelingArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 20
model_args.dataset_type = "simple"
model_args.wandb_project = "geo.ca"
model_args.evaluate_during_training = True
model_args.train_batch_size = 8
model_args.eval_batch_size = 8


In [55]:
# Print the model arguments.
# print(json.dumps(model_args.__dict__, indent=2))
# print(len(model_args.__dict__))

In [56]:
# Set path to the language modelling train file.
train_file = "../data/simpletransformer_lm_train.txt"
test_file = "../data/simpletransformer_lm_test.txt"

In [57]:
use_cuda = torch.cuda.is_available()
model = LanguageModelingModel(
    "bert", "bert-base-uncased", args=model_args, use_cuda=use_cuda
)
print("Running on GPU: {}".format(use_cuda))

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another ar

Running on GPU: True


In [58]:
# Fine tune.
model.train_model(train_file, eval_file=test_file)



Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 of 20:   0%|          | 0/20 [00:26<?, ?it/s]d:\Anaconda\envs\similarity_engine\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 2 of 20:   5%|▌         | 1/20 [03:12<1:00:49, 192.07s/it]

100%|██████████| 715/715 [00:12<00:00, 57.34it/s]

Epoch 4 of 20:  15%|█▌        | 3/20 [09:16<51:16, 180.94s/it]  


100%|██████████| 715/715 [00:17<00:00, 40.04it/s]

Epoch 6 of 20:  25%|██▌       | 5/20 [15:27<45:16, 181.10s/it]

100%|██████████| 715/715 [00:14<00:00, 49.38it/s]

Epoch 8 of 20:  35%|███▌      | 7/20

(20800,
 {'global_step': [1040,
   2000,
   2080,
   3120,
   4000,
   4160,
   5200,
   6000,
   6240,
   7280,
   8000,
   8320,
   9360,
   10000,
   10400,
   11440,
   12000,
   12480,
   13520,
   14000,
   14560,
   15600,
   16000,
   16640,
   17680,
   18000,
   18720,
   19760,
   20000,
   20800],
  'perplexity': [tensor(10.1535),
   tensor(6.6638),
   tensor(6.4761),
   tensor(5.0540),
   tensor(4.8265),
   tensor(4.5530),
   tensor(4.3890),
   tensor(4.1988),
   tensor(3.9632),
   tensor(3.8996),
   tensor(3.7567),
   tensor(3.8552),
   tensor(3.4870),
   tensor(3.5434),
   tensor(3.3936),
   tensor(3.3135),
   tensor(3.3898),
   tensor(3.3866),
   tensor(3.2064),
   tensor(3.2915),
   tensor(3.1971),
   tensor(3.0039),
   tensor(3.1369),
   tensor(3.1712),
   tensor(3.0773),
   tensor(3.1156),
   tensor(3.1087),
   tensor(3.0493),
   tensor(3.0594),
   tensor(3.0970)],
  'eval_loss': [2.317821430342813,
   1.8966902898203941,
   1.8681242404685867,
   1.6201778619039444,

In [16]:
result = model.eval_model(test_file)
wandb.log(result)

{'eval_loss': 2.0003846917991286, 'perplexity': tensor(7.3919)}

## Pytorch / Huggingface fine-tuning code.

Some hyperparameters to consider when using huggingface.

use fp16 to fine-tune.

use the AdamW optimzer.

scheduler: use linear schedule with warmup

batch_size for everything: 8

early stopping True

early stopping patience: 3

gradient_accumulation_steps=1

learning_rate=4e-05

max_grad_norm=1.0, 

max_seq_length=128

warmup_ratio=0.06

warmup_steps=0

strip_accents=True

handle_chinese_characters=True


In [31]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT model
bert_tokenizer = AutoTokenizer.from_pretrained('D:\\similarity-engine\\Jupyter_notebooks\\outputs\\')




Some weights of BertModel were not initialized from the model checkpoint at D:\similarity-engine\Jupyter_notebooks\outputs\ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
bert_model = AutoModel.from_pretrained('bert-base-uncased')

d:\Anaconda\envs\similarity_engine\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
